In [1]:
import sys, os
from polychrom import forcekits, forces, starting_conformations, simulation
from polychrom.hdf5_format import HDF5Reporter
import numpy as np
import openmm as mm
from dataclasses import dataclass
from math import pi
from typing import Dict, List, Tuple, TypedDict, Any
import pickle
import numpy as np
import simtk.unit
from polychrom.forces import tether_particles, pull_force



In [2]:
from poly_shared import (
    MoleculeStructure,
    Bond,
    Angle,
    Dihedral,
    dihedral_angle,
    verify_bond_angle_dihedral_uniqeness,
    concatenate_molecule_structures,
    build_ribbon,
    connect_ribbon_ends,
    connect_with_polymer,
    add_polymer_forces,
    build_hairy_ring,
    build_line_attached_to_monomer,
    threeWayAttraction,
    add_fucking_complicated_force,
    MoleculeStructureCollection,
    attractiveBondForce,
    find_minimum_triplet_distance,
)

In [ ]:
"""
Note: the structures here have starting conformations, 
but the main structure (ribbon) is held by very strong forces, so forces immediately overpower 
the starting conformations. Starting conformation helps the thing "assemble" correctly
(it can fuck itself up if you e.g. start from just a straight "ladder") 
but it basically "assembles" itself fully over the first probably 30-100 timesteps. 

The other goal of the starting conformations is to retain the topology of DNA where it should be - 
if you start from complete garbage, it will jerk too much upon assembly and loose the topology. 
"""

# ----------- Parameters that control the visualization forces -----
# they are there to hold the cohesin "in place". 
# set to False for all realistic simulations
# TODO: pull out the control of everything out of the main blob of code.

pull_ribbon_bottom = False
tether_ribbon_hinge = False

# Holding bonds that are between structures or otherwise added later
extra_bonds: list[Bond] = []
extra_angles: list[Angle] = []
extra_dihedrals: list[Dihedral] = []
non_display_bonds: list[Bond] = []  # bonds that are not displayed in the visualization
extra_kw = {"extra_bonds": extra_bonds, "extra_angles": extra_angles, "extra_dihedrals": extra_dihedrals}

# These effectively became "fixed"  - would be too much to edit them now
l_till_break = 14  # position of the "Elbow" 
n_rows = 28  # length of the cohesin

shared_kwargs: dict[str, Any] = {
    "right_angle_k": 100.0,
    "backbone_angle_k": 150.0,
    "dihedral_k": 100.0,
    "n_rows": n_rows,
    "break_idx": n_rows - l_till_break,
}   # These are very strong forces. They are basically "structural". 


# A very important parameters that controls the amount of twist in a ribbon. 
# If it's wrong, the "elbows" don't align right and the thing has hard time 
# bending from just molecular motion. 
# Also effectively fixed unless you tinker with hinge connection
# Then need to do "free moving" test with no twist dihedrals and possibly adjust this.
d_off = 0.81

# Container to hold our babies
structures = MoleculeStructureCollection()

# Building the "SMC" ribbon
ribbon = build_ribbon(**shared_kwargs, dihedral_offset=d_off, break_sign=1.0)
structures["ribbon1"] = ribbon

# Next structure is always built "starting" with the last index of the previous one 
# Probably a bad convention, but alas.
ribbon2 = build_ribbon(
    start=(0, 3, 0),
    start_particle_index=structures.next_idx(),
    dihedral_offset=-d_off,
    break_sign=-1.0,  # to "mirror" the elbow of the second structure
    **shared_kwargs
)
structures["ribbon2"] = ribbon2

# does not create anything - just edits extra bonds
connect_ribbon_ends(ribbon, ribbon2, **extra_kw, last_row_length=3, penult_row_length=3.3)

# Sorry for the mess. 
# Connection of ribbon ends (=hinge) not displayed - 4 long bonds to keep two ribbons attached
non_display_bonds.extend(extra_bonds[-4:])  

# connect ribbon starts (=heads) with a fixed bond, and make it perpendicular to the backbone
# This is to keep the heads at a distance and also allow twist dihedrals to work
sp1 = ribbon.start_particle_index
sp2 = ribbon2.start_particle_index
extra_bonds.append((sp1, sp2, 4.5, 0.15))
non_display_bonds.append((sp1, sp2, 4.5, 0.15))  # similarly omit from display
extra_angles.append((sp2, sp1, sp1 + 2, pi / 2, shared_kwargs["right_angle_k"]))
extra_angles.append((sp1, sp2, sp2 + 2, pi / 2, shared_kwargs["right_angle_k"]))

# The twist dihedral part that "twists" the heads.
USE_TWIST_DIHEDRALS = False  # whether to use dihedrals to stabilize the twisting of ribbons at the connection
# stabilize the twisting of ribbons at the connection
twist_angle = 2.1  # empirically obtained typical angle at rest
# record dihedral indices for updating them in the future
dih1 = (sp1, sp2, sp2 + 2, sp2 + 2 + 1)
dih2 = (sp2, sp1, sp1 + 2, sp1 + 2 + 1)

extra_dihedrals.append((*dih1, -twist_angle, shared_kwargs["dihedral_k"]/20))
extra_dihedrals.append((*dih2, twist_angle, shared_kwargs["dihedral_k"]/20))

# Connecting the two heads and the hinge with the polymer 
# Hinge is connected to prevent DNA from getting stuck between the coils 
# (while ribbons at hinge are held by bonds, the bonds have no "body" and are transparent)
polymer = connect_with_polymer(  # hinge
    ribbon,
    ribbon.get_end_idx(),
    ribbon2,
    ribbon2.get_end_idx(),
    start_mon_idx=structures.next_idx(),
    n_monomers=6,
    extra_bonds=extra_bonds,
)

# The two polymers below make a topological embrace at the head side of the ribbons 
structures["polymer1"] = polymer
polymer2 = connect_with_polymer(  # heads
    ribbon,
    ribbon.start_particle_index,
    ribbon2,
    ribbon2.start_particle_index,
    start_mon_idx=structures.next_idx(),
    n_monomers=13,
    extra_bonds=extra_bonds,
)
structures["polymer2"] = polymer2

polymer3 = connect_with_polymer(  # also heads, but up the structure
    ribbon,
    ribbon.start_particle_index + 5,
    ribbon2,
    ribbon2.start_particle_index + 5,
    start_mon_idx=structures.next_idx(),
    n_monomers=8,
    extra_bonds=extra_bonds,
)
structures["polymer3"] = polymer3


# Building the NIPBL and the hinge stick structures
# Size of nipbl was empirically adjusted
nipbl1 = build_line_attached_to_monomer(
    polymer2,
    polymer2.start_particle_index + 3,
    n_monomers=27,
    start_mon_idx=structures.next_idx(),
    bond_dl=0.05,
    extra_bonds=extra_bonds,
)
structures["nipbl1"] = nipbl1

## scrunch up the NIPBL a bit by adding random weak bonds   -  was abandoned
# for i in range(0, len(nipbl1.positions) - 6, 3):
#    extra_bonds.append((nipbl1.start_particle_index + i, nipbl1.start_particle_index + i + 5, 0.5, 1.8))
#    non_display_bonds.append((nipbl1.start_particle_index + i, nipbl1.start_particle_index + i + 5, 0.5, 1.8))

# A tiny short "hinge stick" 
hinge_stick1 = build_line_attached_to_monomer(
    polymer,
    polymer.start_particle_index + 3,
    n_monomers=2,
    start_mon_idx=structures.next_idx(),
    bond_dl=0.05,
    extra_bonds=extra_bonds,
)
structures["hinge_stick1"] = hinge_stick1



ring = build_hairy_ring(
    radius=50,
    n_monomers=420,  # a little more than 2*pi*50 because MD does not like straight lines,
    plane="XZ",
    p_hairy=0.5,
    seed=42,
    backbone_k=5.0,
    start_mon_idx=structures.next_idx(),
)
structures["ring"] = ring

# ring is build in XZ plane
# ring needs to pass through (1.5, 0, 1.5) to be in the right place
ring.positions = ring.positions - np.array([0, 0, 50])  # shift the ring to pass through (0,0,0)
ring.positions = ring.positions + np.array([1.5, 0, 1.5])  # shift the ring to pass through (1.5, 0, 1.5)



# These indices will be used to find the triplet with minimum distance
# and apply three-way attraction force to it

# The triplet consists of:
# - last two monomers of each hinge stick (type 1)
# - last two monomers of each nipbl (type 2)
# - the entire ring (type 3)

# type 1 - last 2 monomers of each hinge stick
type1_indices = [hinge_stick1.start_particle_index + len(hinge_stick1.positions) - 1]

# type 2 - last 2 monomers of each nipbl
type2_indices = [nipbl1.start_particle_index + len(nipbl1.positions) - 1]

# type 3 indices - the entire ring
type3_indices = list(range(ring.start_particle_index, ring.start_particle_index + len(ring.positions) - 2))


combined_structure = concatenate_molecule_structures(structures)

N = combined_structure.positions.shape[0]
positions = combined_structure.positions

bonds = combined_structure.bonds + extra_bonds
angles = combined_structure.angles + extra_angles
dihedrals = combined_structure.dihedrals + extra_dihedrals


# positions = pickle.load(open("captured_try1.pkl", "rb"))
# positions = pickle.load(open("start_try1.pkl", "rb"))

# -------- creating the simulation objects ------------------
reporter = HDF5Reporter(folder="trajectory", max_data_length=5, overwrite=True)
final_collision_rate = 0.01
sim = simulation.Simulation(
    platform="CUDA",
    integrator="variableLangevin",
    error_tol=0.001,
    GPU="0",
    collision_rate=0.01,
    N=N,
    save_decimals=5,
    PBCbox=False,
    reporters=[reporter],
)


sim.set_data(positions, center=False)  # loads a polymer, does not center anything. 

# Add forces to the simulation
res = add_polymer_forces(sim, bonds, angles, dihedrals)
assert res is not None  # here it won't be None as we have dihedrals
DihedralIndDict, dihedral_force = res

# nonbonded force  -  weak global attraction. 
# Didn't know we even had it. It probably doesn't hurt, can push it a little more 
# towards coil-globule transition 
rep_force = forces.smooth_square_well(sim, 30, 1.03, attractionEnergy=0.35, attractionRadius=1.5)
sim.add_force(rep_force)


# tether_idx = list(range(ring.start_particle_index, ring.get_next_idx()))
# tether_pos = ring.positions * 0.8 
# len(tether_idx), len(tether_pos)
# tether_force = forces.tether_particles(sim, tether_idx, k=0.1, positions=list(tether_pos))
# sim.add_force(tether_force)

# TODO: should we re-enable this? or it doesn't matter? 
#remover = mm.CMMotionRemover()
#remover.name = "remover"  # type: ignore - this is polychrom's old convention
#sim.add_force(remover)  # add motion remover to remove center of mass motion

# Add a fixed force to close the hinge 
k_linear = 2.2

# add a fucking complicated force  (True/False switch was used to allow it to "flop" naturally without twist dihedrals on)
if True: 
    fuckingComplicatedForce = add_fucking_complicated_force(sim, k_linear)
    break_idx = shared_kwargs["break_idx"]
    for ribbon_use, cpl_sign, brk_sign in ((ribbon, 1, 1), (ribbon2, -1, -1)):
        # add particles in the order they are used in the expression

        particles = [ribbon_use.index_dict[(i, j)] for i in range(break_idx - 3, break_idx + 2) for j in (0, 1)]
        print(particles)
        fuckingComplicatedForce.addBond(particles, [cpl_sign, brk_sign])

    sim.add_force(fuckingComplicatedForce)

# Three-way attraction force 

# This is the force that is suuper delicate to what the substrate is. Sometimes adding 0.4 will make it "stick forever"  
# And subtracting will make it detach immediately
threeway_cutoff = 1.8  
threeway_strength = 4.6

three_way_force = threeWayAttraction(
    sim,
    type1_particle_idx=type1_indices,
    type2_particle_idx=type2_indices,
    type3_particle_idx=type3_indices,
    attractionEnergy=threeway_strength,
    attractionRadius=threeway_cutoff, 
)
sim.add_force(three_way_force)

# A little "helper" force to make type1 and type2 (I think NipBL and hinge stick) 
# stick to each other a little - attempt to make capture more robust 
# (not sure it really helped, but kept for historical force) 
type12_bonds = [(i, j) for i in type1_indices for j in type2_indices]
attr_force = attractiveBondForce(sim, type12_bonds, strength_kt=4.1, cutoff=3.5)
sim.add_force(attr_force)

# Visualization force to keep cohesin stationary and "suspended"
# Please disable for any real run. 
idx1, idx2 = ribbon.start_particle_index, ribbon.get_end_idx()
idx3 = (idx1 + idx2)//2 

idx4, idx5 = ribbon2.start_particle_index, ribbon2.get_end_idx()
idx6 = (idx4 + idx5) // 2 

if pull_ribbon_bottom: 
    pf = pull_force(sim, (idx1, idx2, idx3, idx4), [[0.05,-0.4, 0], [-0.05, -0.4, 0]] * 2)
    sim.add_force(pf)

if tether_ribbon_hinge:
    idx_hinge = ribbon.start_particle_index + 2 * l_till_break
    idx_hinge2 = ribbon2.start_particle_index + 2 * l_till_break
    tether_force = tether_particles(sim, (idx_hinge,idx_hinge2), k=3)
    tether_force.name = "Tether2"
    sim.add_force(tether_force)


# ------------ Start of the simulation ----------

# Just do some blocks of energy minimization, saving in between. 
sim.eK_critical = 3000
sim.do_block(0, save_extras={"bonds": bonds})
sim.local_energy_minimization()
sim.do_block(0, save_extras={"bonds": bonds})
sim.local_energy_minimization()
sim.do_block(0, save_extras={"bonds": bonds})
sim.local_energy_minimization()
sim.do_block(0, save_extras={"bonds": bonds})


bonds_to_save = [i for i in bonds if i not in non_display_bonds]
# the exclusion code looked fragile, just adding a check that _something_ is excluded 
if len(bonds) == len(bonds_to_save):
    raise ValueError("Something went wrong, all bonds are displayed, but some should not be!")

all_dh = []
cooldown = -1
# This number is basically "run forever" 
for block in range(1000000):
    # pickling the structures and saving to HDF5 - requires casting to numpy array
    s_pkl = np.frombuffer(pickle.dumps(structures), dtype=np.uint8)
    do_print = block % 20 == 0
    # First blocks are tiny to save the trajectory of going into this
    sim.do_block(
        10 * block if block < 50 else 2000,
        save_extras={"bonds": bonds_to_save, "structures": s_pkl},
        print_messages=do_print,
    )
    if block == 50:  # re-set collision rate to the final value
        sim.collisionRate = final_collision_rate * (1 / simtk.unit.picosecond)  # type: ignore
        sim.integrator.setFriction(sim.collisionRate)  # type: ignore

    pos = sim.get_data()

    # Calculating twist dihedrals and distance for statistics on that
    dh1 = dihedral_angle(pos, sp1, sp2, sp2 + 2, sp2 + 2 + 1)
    dh2 = dihedral_angle(pos, sp2, sp1, sp1 + 2, sp1 + 2 + 1)
    dist = np.sum((pos[ribbon.start_particle_index] - pos[ribbon.get_end_idx()]) ** 2)
    # print(f"Block {block}: dihedral angles {dh1:.2f}, {dh2:.2f}, distance {dist:.2f}")
    all_dh.append((dh1, dh2, dist))

    triplet_dist, _, num_triplets = find_minimum_triplet_distance(
        pos, type1_indices, type2_indices, type3_indices, threeway_cutoff
    )

    type12_dist = np.linalg.norm(pos[type1_indices][:, None, :] - pos[type2_indices][None, :, :], axis=-1).min()

    # ------ Activating / Deactivating the twist dihedrals --------- 

    # These will start "twisting" the heads upon the 3-way contact formation.
    # turning dihedrals "on" upon the distance becoming small enough
    if triplet_dist < threeway_cutoff * 0.8 and not USE_TWIST_DIHEDRALS:
        if cooldown == -1:  # if cooldown is not set
            cooldown = 20  # set cooldown to 20 blocks
        if cooldown == 0:
            USE_TWIST_DIHEDRALS = True
            twist_angle = 2.1  # reset twist angle to the initial value
            print("Turning on twist dihedrals")
            cooldown = -1  # reset cooldown

    if triplet_dist > threeway_cutoff * 1.3 and USE_TWIST_DIHEDRALS:
        if cooldown == -1:
            cooldown = 20  # set cooldown to 10 blocks
        if cooldown == 0:
            USE_TWIST_DIHEDRALS = False
            twist_angle = 0.0  # reset twist angle to 0
            print("Turning off twist dihedrals")
            cooldown = -1  # reset cooldown
    cooldown -= 1
    if cooldown < -1:
        cooldown = -1

    print("\r", end="")
    print(triplet_dist, num_triplets, type12_dist, cooldown, USE_TWIST_DIHEDRALS, end="; ")

    # ---------- Actually updating twist dihedrals -------------
    ind1 = DihedralIndDict[dih1]
    ind2 = DihedralIndDict[dih2]
    if USE_TWIST_DIHEDRALS:
        twist_angle = max(twist_angle - 0.01, 0)

        dihedral_force.setTorsionParameters(ind1, *dih1, (-twist_angle, shared_kwargs["dihedral_k"]))
        dihedral_force.setTorsionParameters(ind2, *dih2, (twist_angle, shared_kwargs["dihedral_k"]))
        dihedral_force.updateParametersInContext(sim.context)  # update the parameters in the context
    else:  # basically invalidate the dihedrals, so that they do not affect the simulationd
        # Set k for a very very tiny number 
        dihedral_force.setTorsionParameters(ind1, *dih1, (-twist_angle, shared_kwargs["dihedral_k"] / 1000))
        dihedral_force.setTorsionParameters(ind2, *dih2, (twist_angle, shared_kwargs["dihedral_k"] / 1000))
        dihedral_force.updateParametersInContext(sim.context)  # update the parameters in the context


sim.print_stats()  # In the end, print very simple statistics

reporter.dump_data()  # always need to run in the end to dump the block cache to the disk

In [ ]:
# --------Below is just debug garbage---------- 

In [ ]:
cooldown

In [ ]:
dh = np.array(all_dh)
np.mean(dh, axis=0)  # mean dihedral angles

In [ ]:

import matplotlib.pyplot as plt
plt.figure() 
plt.scatter(dh[:, 0], dh[:, 2], s=1,  cmap="viridis")

In [ ]:
pos1 = pos[type1_indices]  # shape: (n_type1, 3)
pos2 = pos[type2_indices]  # shape: (n_type2, 3)
pos3 = pos[type3_indices]  # shape: (n_type3, 3)

min_distance = np.inf
best_triplet = np.array((1, 1, 1))  # Initialize with dummy values

# Loop over type1 and type2 (small arrays)
for i1, idx1 in enumerate(type1_indices):
    for i2, idx2 in enumerate(type2_indices):
        # Vectorized distance calculation for all type3
        d12 = np.linalg.norm(pos1[i1] - pos2[i2])        
        d13 = np.linalg.norm(pos1[i1] - pos3, axis=1)  # vectorized over type3
        d23 = np.linalg.norm(pos2[i2] - pos3, axis=1)  # vectorized over type3

        # Maximum distance for each type3 particle with this (type1, type2) pair
        triplet_min_distances = np.maximum(d12, np.maximum(d13, d23))

        # Find the best type3 for this (type1, type2) pair
        best_i3 = np.argmin(triplet_min_distances)
        current_min = triplet_min_distances[best_i3]

        if current_min < min_distance:
            min_distance = current_min
            best_triplet = np.array((idx1, idx2, type3_indices[best_i3]))


min_distance

In [ ]:
dh1, dh2

In [ ]:
pos3